## Notebook for Imputing C13k 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy import stats

import torch

In [46]:
c13k = pd.read_csv('c13k_selections.csv')
c13k.head()

,Problem,Feedback,n,Block,Ha,pHa,La,Hb,pHb,Lb,LotShapeB,LotNumB,Amb,Corr,bRate,bRate_std
0,1,True,15,2,26,0.95,-1,23,0.05,21,0,1,False,0,0.626667,0.384460
1,2,True,15,4,14,0.60,-18,8,0.25,-5,0,1,True,-1,0.493333,0.413118
2,3,True,17,4,2,0.50,0,1,1.00,1,0,1,False,0,0.611765,0.432843
3,4,True,18,3,37,0.05,8,87,0.25,-31,1,2,False,0,0.222222,0.387383
4,5,False,15,1,26,1.00,26,45,0.75,-36,2,5,False,0,0.586667,0.450185


In [61]:
from random import sample
import random

random.seed(1234)

pH_set=[.01,.05,.1,.2,.25,.4,.5,.6,.75,.8,.9,.95,.99, 1]
tmp=[-7,-6,-5,-4,-3,-2,2,3,4,5,6,7,8]
def LotteryA(c13k):
    pH_set = [.01, .05, .1, .2, .25, .4, .5, .6, .75, .8, .9, .95, .99, 1]
    tmp = [-7, -6, -5, -4, -3, -2, 2, 3, 4, 5, 6, 7, 8]
    
    EVA_list = []
    HA_list = []
    pHA_list = []
    LA_list = []
    LotNumA_list = []
    LotShapeA_list = []

    # Generate rows matching the length of c13k
    for _ in range(len(c13k)):
        EVA = sample([i for i in range(-10, 31)], 1)[0]
        LA, HA = c13k[['La', 'Ha']].iloc[_]
        pHA = c13k['pHa'].iloc[_]
        LotNumA = 1
        LotShapeA = 0

        temp = np.random.triangular(-50, EVA, 120, size=None)

        p = np.random.uniform(0, 1)
        if p <= .6:
            LotNumA = 1
            LotShapeA = '-'
        elif p > .6 and p <= .8:
            temp = sample(tmp, 1)[0]
            LotNumA = abs(temp)
            if temp > 0:
                LotShapeA = 'R-skew'
            else:
                LotShapeA = 'L-skew'
        else:
            LotNumA = sample([3, 5, 7, 9], 1)[0]
            LotShapeA = 'Symm'
            
        EVA_list.append(EVA)
        HA_list.append(HA)
        pHA_list.append(pHA)
        LA_list.append(LA)
        LotNumA_list.append(LotNumA)
        LotShapeA_list.append(LotShapeA)
        

    # Create DataFrame from lists
    # lottery_df = pd.DataFrame({
    #     'EVA': EVA_list,
    #     'Ha': HA_list,
    #     'pHa': pHA_list,
    #     'La': LA_list,
    #     'LotNumA': LotNumA_list,
    #     'LotShapeA': LotShapeA_list
    # })
    c13k['EVA'] = EVA_list
    c13k['LotShapeA'] = LotShapeA_list
    c13k['LotNumA'] = LotNumA_list

    return c13k


# Generate the DataFrame matching the length of c13k
lottery_df = LotteryA(c13k)


In [70]:
lottery_df.loc[lottery_df['LotShapeB'] == 0, 'LotShapeB'] = '-'   # Replace values in column 'B' with 'fruit' where 'A' < 3
lottery_df.loc[lottery_df['LotShapeB'] == 1, 'LotShapeB'] = 'Symm'  # Replace values in column 'B' with 'vegetable' where 'A' >= 3
lottery_df.loc[lottery_df['LotShapeB'] == 2, 'LotShapeB'] = 'R-skew' 
lottery_df.loc[lottery_df['LotShapeB'] == 3, 'LotShapeB'] = 'L-skew' 

lottery_df.loc[lottery_df['Amb'] == False, 'Amb'] = '0' 
lottery_df.loc[lottery_df['Amb'] == True, 'Amb'] = '1' 

/var/folders/51/42w_kz156q920kx3w9jz1lwm0000gn/T/ipykernel_51163/2433339887.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  lottery_df.loc[lottery_df['Amb'] == False, 'Amb'] = '0'


In [71]:
lottery_df

,Problem,Feedback,n,Block,Ha,pHa,La,Hb,pHb,Lb,LotShapeB,LotNumB,Amb,Corr,bRate,bRate_std,EVA,LotShapeA,LotNumA
0,1,True,15,2,26,0.95,-1,23,0.05,21,-,1,0,0,0.626667,0.384460,18,Symm,3
1,2,True,15,4,14,0.60,-18,8,0.25,-5,-,1,1,-1,0.493333,0.413118,-10,-,1
2,3,True,17,4,2,0.50,0,1,1.00,1,-,1,0,0,0.611765,0.432843,-5,-,1
3,4,True,18,3,37,0.05,8,87,0.25,-31,Symm,2,0,0,0.222222,0.387383,27,L-skew,7
4,5,False,15,1,26,1.00,26,45,0.75,-36,R-skew,5,0,0,0.586667,0.450185,-5,L-skew,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14563,13002,True,15,3,30,1.00,30,42,0.80,0,-,1,1,0,0.367619,0.302731,-1,-,1
14564,13003,True,15,5,70,0.50,-42,18,0.80,7,-,1,0,0,0.760000,0.364104,9,-,1
14565,13004,True,15,5,8,0.40,-17,31,0.40,-34,Symm,6,0,0,0.666667,0.367747,6,-,1
14566,13005,True,15,2,89,0.50,-49,45,0.50,-12,-,1,0,0,0.386667,0.381476,12,-,1


## write df to csv


In [74]:
lottery_df.to_csv('c13k_imputed.csv', index=False)  